In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time


In [4]:
# 드라이버 종료
driver.quit()

## team-box-score - playoffs

In [5]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
'''
# 크롬 옵션 설정
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")  
options.add_argument("--disable-dev-shm-usage")  
options.add_argument("--window-size=1920,1080")
options.add_argument("--remote-debugging-port=9223")  # 다른 포트로 설정

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
'''

driver = webdriver.Chrome()

# 크롤링할 시즌 목록 (2000-01 시즌부터 2022-23시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2014, 2023)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    url = f"https://www.nba.com/stats/teams/boxscores?Season={season}&SeasonType=Playoffs"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업 닫기 시도 (있을 경우)
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 최대 페이지 설정
    max_pages = 5

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

    # 페이지 반복 처리
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        # 테이블을 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        #열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:25]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')
        if not rows:
            print(f"No data rows found for season {season}.")
            break

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][0:]
            data.append(season)  # 시즌 정보 추가
            
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지로 이동
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break
            next_button.click()
            time.sleep(5)  # 페이지 로드 대기
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    globals()[f'all_data_{season}'] = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")
    globals()[f'all_data_{season}'].to_csv(f'team_boxscore/team_boxscore_{season}.csv',index=False)
    print(f"Season {season} data saved.")

# 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
#print(all_data.head())


No cookie popup to close: Message: 
Stacktrace:
0   chromedriver                        0x0000000103214500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x000000010320c890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000102e1c538 cxxbridge1$string$len + 89424
3   chromedriver                        0x0000000102e60878 cxxbridge1$string$len + 368784
4   chromedriver                        0x0000000102e9ab7c cxxbridge1$string$len + 607124
5   chromedriver                        0x0000000102e55374 cxxbridge1$string$len + 322444
6   chromedriver                        0x0000000102e55fc4 cxxbridge1$string$len + 325596
7   chromedriver                        0x00000001031dbd2c cxxbridge1$str$ptr + 1685732
8   chromedriver                        0x00000001031e0530 cxxbridge1$str$ptr + 1704168
9   chromedriver                        0x00000001031c0e08 cxxbridge1$str$ptr + 1575360
10  chromedriver                        0x00000001031e0e00 cxxb